In [148]:
from search import ( # Bases para construcción de problemas
    Problem, Node, Graph, UndirectedGraph,
    SimpleProblemSolvingAgentProgram,
    GraphProblem
)

from search import ( # Algoritmos de búsqueda no informada
    tree_search, graph_search, best_first_graph_search,
    breadth_first_tree_search, breadth_first_search,
    recursive_best_first_search,
    depth_first_tree_search, depth_first_graph_search,
    depth_limited_search, iterative_deepening_search,
    uniform_cost_search,
    compare_searchers
)

from search import ( # Algoritmos de búsqueda informada (heurística)
    greedy_best_first_graph_search, astar_search
)

from math import sqrt
import statistics

In [139]:
class BrokenCalc(Problem):
    """The abstract class for a formal problem.  You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""
    def __init__(self, initial = 0, goal=0, operators=["+", "*"], operands=[2, 3]):
        Problem.__init__(self, initial, goal)
        self.operands = operands
        self.operators = operators
        
        self.ops = [(operand, operator) for operand in operands for operator in operators]
        self.ops = self.ops + [(operand, 'press') for operand in operands]

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        available = list(self.ops)
        
        # Remove SQRT if the current state is a negative number
        if state < 0:
            available = list(filter(lambda x: x[1] != 'sqrt', available))
        
        return available

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        funcs = {
            '+': lambda x: state + x,
            '*': lambda x: state * x,
            '/': lambda x: state / x,
            '-': lambda x: state - x,
            'sqrt': lambda _: sqrt(state),
            'press': lambda x: (state * 10) + x
        }
        return funcs[action[1]](action[0])
    def h(self, node):
        "Diferencia entre meta y estado actual"
        return abs(self.goal - node.state)
    def h2(self, node):
        if node.state != 0:
            return (self.goal % node.action[0]) + node.action[0]
        else:
            return 0

#     def path_cost(self, c, state1, action, state2):
#         """Return the cost of a solution path that arrives at state2 from
#         state1 via action, assuming cost c to get up to state1. If the problem
#         is such that the path doesn't matter, this function will only look at
#         state2.  If the path does matter, it will consider c and maybe state1
#         and action. The default method costs 1 for every step in the path."""
#         return c + 1

#     def value(self, state):
#         """For optimization problems, each state has a value.  Hill-climbing
#         and related algorithms try to maximize this value."""
#         raise NotImplementedError

# print_solution
This method prints the solution to a problem. It just concatenates each action in the format `<OP> <N>` giving it as a result a string like:

$$ 0 \; OP_1 \; N_1 \; OP_2 \; N_2 \ldots = GOAL$$

Where $OP_i \in \{ +, -, *, sqrt \}$ and $N_i \in \mathbb{R}$

In [140]:
def print_solution(problem, goal):
    path = goal.solution()
    print("0", end = '')
    ops = "".join(map(lambda action: " {} {}".format(action[1], action[0]), path))
    print(ops, end = '')
    *_, res = goal.path()
    print(" = %s"  % res.state)
#     print(" ==> COST: %d" % goal.path_cost)

In [141]:
p1 = BrokenCalc(goal = 20, operators=["+", "*"], operands=[2, 3])
goal1 = breadth_first_search(p1)
print("PROBLEM actions: %s" % p1.ops)
print("SOLUTION: %s" % goal1.solution())
print("PATH: %s" % goal1.path())

print_solution(p1, goal1)

PROBLEM actions: [(2, '+'), (2, '*'), (3, '+'), (3, '*'), (2, 'press'), (3, 'press')]
SOLUTION: [(2, '+'), (3, '+'), (2, '*'), (2, '*')]
PATH: [<Node 0>, <Node 2>, <Node 5>, <Node 10>, <Node 20>]
0 + 2 + 3 * 2 * 2 = 20


In [142]:
level_1_nums = [2, 3]
level_3_nums = [1, 6, 8]

level_1_operators = ["+", "*"]
level_3_operators = ["-", "sqrt"]

level_1_goals = [6, 7, 8, 10, 12, 15, 20, 50]
level_3_goals = [-5, 3, 5, 13, 20, 33, 82, 100]

# Level 1 solutions
__5) Execute the blind and heuristic search methods (with the 2 heuristics) to solve the selected level for 3 numbers that can be calculated and 3 numbers that cannot, and show
the solutions found by each method.__

The following code solves all the numbers given in the assignment description for Level 1. 

## BFS

In [143]:
print("The following numbers can be calculated: %s" % level_1_goals )
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = breadth_first_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

The following numbers can be calculated: [6, 7, 8, 10, 12, 15, 20, 50]
Solving for 6
0 + 2 * 3 = 6
Solving for 7
0 + 2 + 2 + 3 = 7
Solving for 8
0 + 2 + 2 * 2 = 8
Solving for 10
0 + 2 + 3 * 2 = 10
Solving for 12
0 + 2 + 2 * 3 = 12
Solving for 15
0 + 2 + 3 * 3 = 15
Solving for 20
0 + 2 + 3 * 2 * 2 = 20
Solving for 50
0 + 2 press 2 + 3 * 2 = 50
COSTS: [2, 3, 3, 3, 3, 3, 4, 4] | AVG: 3.125000


## DFS

In [146]:
print("The following numbers can be calculated: %s" % level_1_goals )
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = depth_limited_search(p, limit = 10)
    print("GOALS")
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

The following numbers can be calculated: [6, 7, 8, 10, 12, 15, 20, 50]
Solving for 6
GOALS
0 + 2 + 2 + 2 = 6
Solving for 7
GOALS
0 + 2 + 2 + 3 = 7
Solving for 8
GOALS
0 + 2 + 2 + 2 + 2 = 8
Solving for 10
GOALS
0 + 2 + 2 + 2 + 2 + 2 = 10
Solving for 12
GOALS
0 + 2 + 2 + 2 + 2 + 2 + 2 = 12
Solving for 15
GOALS
0 + 2 + 2 + 2 + 2 + 2 + 2 + 3 = 15
Solving for 20
GOALS
0 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 = 20
Solving for 50
GOALS
0 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 * 3 + 2 = 50
COSTS: [3, 3, 4, 5, 6, 7, 10, 10] | AVG: 6.000000


## Uniform Cost Search

In [116]:
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = uniform_cost_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for 6
0 + 2 * 3 = 6
Solving for 7
0 + 2 + 2 + 3 = 7
Solving for 8
0 + 2 + 2 * 2 = 8
Solving for 10
0 + 2 + 3 * 2 = 10
Solving for 12
0 + 2 + 2 * 3 = 12
Solving for 15
0 + 2 + 3 * 3 = 15
Solving for 20
0 + 2 + 3 * 2 * 2 = 20
Solving for 50
0 + 2 press 2 + 3 * 2 = 50
COSTS: [2, 3, 3, 3, 3, 3, 4, 4] | AVG: 3.125000


## Iterative Deepening Search

In [117]:
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = iterative_deepening_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for 6
0 + 2 * 3 = 6
Solving for 7
0 + 2 + 2 + 3 = 7
Solving for 8
0 + 2 + 2 * 2 = 8
Solving for 10
0 + 2 + 3 * 2 = 10
Solving for 12
0 + 2 + 2 * 3 = 12
Solving for 15
0 + 2 + 3 * 3 = 15
Solving for 20
0 + 2 + 3 * 2 * 2 = 20
Solving for 50
0 + 2 press 2 + 3 * 2 = 50
COSTS: [2, 3, 3, 3, 3, 3, 4, 4] | AVG: 3.125000


## A* search using h

In [118]:
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = astar_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for 6
0 + 3 * 2 = 6
Solving for 7
0 + 3 + 2 + 2 = 7
Solving for 8
0 + 3 * 2 + 2 = 8
Solving for 10
0 + 3 * 2 + 2 + 2 = 10
Solving for 12
0 + 3 * 3 + 3 = 12
Solving for 15
0 + 3 * 3 + 3 + 3 = 15
Solving for 20
0 + 3 * 3 * 2 + 2 = 20
Solving for 50
0 + 3 press 3 + 3 + 3 + 3 + 3 + 3 + 2 = 50
COSTS: [2, 3, 3, 4, 3, 4, 4, 8] | AVG: 3.875000


## A* search using h2

In [120]:
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = astar_search(p, p.h2)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for 6
0 + 3 * 2 = 6
Solving for 7
0 + 3 + 2 + 2 = 7
Solving for 8
0 + 2 + 2 * 2 = 8
Solving for 10
0 + 3 + 2 * 2 = 10
Solving for 12
0 + 3 * 2 * 2 = 12
Solving for 15
0 + 2 + 3 * 3 = 15
Solving for 20
0 + 3 + 2 * 2 * 2 = 20
Solving for 50
0 + 2 press 3 + 2 * 2 = 50
COSTS: [2, 3, 3, 3, 3, 3, 4, 4] | AVG: 3.125000


In [121]:
# LEVEL 3
costs = []
for g in level_3_goals:
    p = BrokenCalc(goal = g, operators = level_3_operators, operands = level_3_nums)
    print("Solving for %s" % g)
    goal = breadth_first_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for -5
0 press 1 - 6 = -5
Solving for 3
0 press 1 press 1 - 8 = 3
Solving for 5
0 press 6 - 1 = 5
Solving for 13
0 press 8 - 6 press 1 - 8 = 13
Solving for 20
0 press 8 - 6 press 1 - 1 = 20
Solving for 33
0 press 1 press 6 sqrt 1 press 1 - 8 = 33.0
Solving for 82
0 press 8 press 8 - 6 = 82
Solving for 100
0 press 1 press 1 - 1 press 1 - 1 = 100
COSTS: [2, 3, 2, 4, 4, 5, 3, 5] | AVG: 3.500000


In [149]:
costs = []
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = recursive_best_first_search(p)
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

Solving for 6
0 + 2 + 2 + 2 = 6
Solving for 7
0 + 2 + 2 + 3 = 7
Solving for 8
0 + 2 + 2 + 2 + 2 = 8
Solving for 10
0 + 2 + 2 + 2 + 2 + 2 = 10
Solving for 12
0 + 2 + 2 + 2 + 2 + 2 + 2 = 12
Solving for 15
0 + 2 + 2 + 2 + 2 + 2 + 2 + 3 = 15
Solving for 20
0 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 = 20
Solving for 50
0 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 = 50
COSTS: [3, 3, 4, 5, 6, 7, 10, 25] | AVG: 7.875000


## Numbers unable to find

In [166]:
impossible_goals = [-10, -20, -1]
print("The following numbers cannot be calculated: %s" % impossible_goals)
costs = []
for g in impossible_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = depth_limited_search(p, limit = 6)
    if not goal or type(goal) == 'str':
        print("NOT FOUND")
        break
    print_solution(p, goal)
    costs.append(goal.path_cost)
print("COSTS: %s | AVG: %f" % (costs, statistics.mean(costs)))

The following numbers cannot be calculated: [-10, -20, -1]
Solving for -10


StatisticsError: mean requires at least one data point